In [ ]:
!python -V

# <center> GENERADORES</center>
# <center> YIELD vs RETURN</center>

In [ ]:
def iterador_generadores(generador):

    while generador:        
        try:
            print(next(generador))
        except StopIteration:
            print('Finalizo iteracion')
            break

def lenguaje():
    yield 'Python'
    yield 'Java'
    yield 'Scala'
               
print(list(lenguaje()))

In [ ]:
iterador_generadores(lenguaje())

# <center> YIELD FROM</center>

In [ ]:
def lista_generador():
    
    yield from 'JorgeCardona'
    
iterador_generadores(lista_generador())

# <center> **YIELD & YIELD FROM** DENTRO DE OTRO YIELD</center>

In [ ]:
def astronomia():
    yield 'Sol'
    yield 'Luna'
    yield 'Estrellas'
    yield 'Galaxias'
    yield 'Planetas'   

def numeros():
    yield 'Uno'
    yield 'Dos'
    yield lenguaje()
    yield from astronomia()
    yield 'Tres'
    yield 'Cuatro'
    yield lista_generador()
    yield 'Cinco'


print(list(numeros()))

# <center> **.\__\___next\__\___()**</center>

In [ ]:
def iterador_tareas(lista_funciones):    
    # itera la lista de funciones con generadores
    while lista_funciones:
        
        print(lista_funciones, len(lista_funciones))
        
        try:
            # obtiene la funcion que tiene generadores
            actual = lista_funciones.pop(0)
            print(actual.__next__())                     
        except Exception:
            # para evaluar si es de tipo Generator
            import types
            print(f"{actual} -> {'Generador sin elementos' if isinstance(actual, types.GeneratorType) else 'No es una expresion Generadora'} , es una  {type(actual)}")
            pass
        else:
            lista_funciones.append(actual)
            
        
        
iterador_tareas([lista_generador(), lenguaje(), astronomia(), numeros()])

# <center> **ENVIAR VALORES A UN GENERADOR**</center>

In [ ]:
def multiplicar_valor():
    while True:
        x,y = yield
        yield x + y
gen = multiplicar_valor()

In [ ]:
for i in range(10,20):
    next(gen)
    x = i//2
    y = i
    
    resultado = gen.send([x,y])
    
    print(f'el valor de {x} + {y} es {resultado}')

# <center> **ENVIAR VALORES A UN GENERADOR CON CONDICIONALES**</center>

In [ ]:
def multiplicar_valor_impar(generador):
    
    while generador:
        # obtiene el valor actual del iterador en el generador
        actual = next(generador)
        print(f'El valor del iterador es {actual}')
        
        # si el valor del generador no es par se le envian parametros
        if actual%2 !=-0:
            
            # parametros a capturar con el metodo send
            x,y = yield
            
            # define las variables a capturar
            if (x and y) is not None:
                yield f'el valor de ({x} x {actual}) + ({y} x {actual} x {2}) es ({x * actual} + {y * actual * 2}) = {x*actual + y*actual*2}'
                print()
        else:
            print(f'El valor {actual} -->, no es un numero impar')
            print()
                
# define el numero maximo de objetos generadores creados
iterador = (i for i in range(10))
gen = multiplicar_valor_impar(iterador)

In [ ]:
# rango superior al total de objetos generadores
for i in range(99):
    
    try:
        next(gen)
    except Exception:
        print('Iteracion Finalizada')
        break
    else:
        resultado_parametros = gen.send([i+2,i+5])
        print(resultado_parametros)

# <center> **EXCEPCIONES EN UN GENERADOR**</center>

In [ ]:
numeros = (valor for valor in range(10))

def iterador_generadores(generador):

    while generador:        
        try:
            
            valor = next(generador)
            print(valor)
            
            # criterio de parada
            if valor > 5:
                generador.throw(ValueError("EL VALOR HA SUPERADO EL LIMITE PERMITIDO"))
                
        except StopIteration:
            print('Finalizo iteracion')
            break
            
iterador_generadores(numeros)

# <center> **DETENER O FINALIZAR UN GENERADOR**</center>

In [ ]:
numeros = (valor for valor in range(10))

def iterador_generadores(generador):

    while generador:        
        try:            
            valor = next(generador)               
        except StopIteration:
            print('Finalizo iteracion')
            break
        else:
            # criterio de parada
            if valor >= 5:
                generador.close()
            print(valor) 

iterador_generadores(numeros)

# <center> **PIPE O CANALIZACIONES CON GENERADORES**</center>

In [ ]:
def fibonacci(nums):
    x, y = 0, 1
    for _ in range(nums):
        x, y = y, x+y
        yield x

def cuadrado(nums):
    for num in nums:
        yield num**2

print(list(fibonacci(10)))
print(list(cuadrado(fibonacci(10))))
print(sum(cuadrado(fibonacci(10))))

# <center> **CONSUMO DE MEMORIA EN BYTES DE UNA LISTA VS UN GENERADOR**</center>

In [ ]:
import sys
lista = [i * 2 for i in range(10000)]
print(sys.getsizeof(lista))

In [ ]:
generador = (i ** 2 for i in range(10000))

while generador: 

    try:
        siguiente = next(generador)
        print(f'tamano del valor generado {sys.getsizeof(siguiente)}, tamano del generador {sys.getsizeof(generador)}, valor {siguiente}')
    except StopIteration:
        print('Finalizo iteracion')
        break

# <center> **VELOCIDAD DE PROCESAMIENTO VS CONSUMO DE MEMORIA DE UNA LISTA VS UN GENERADOR**</center>

In [ ]:
import cProfile

cProfile.run('sum([i * 2 for i in range(10000)])')

In [ ]:
cProfile.run('sum((i * 2 for i in range(10000)))')

# <center> iter()</center>

In [ ]:
iterador = iter([1,2,3,4,5,6,7,8,9,0])

In [ ]:
print(dir(iterador))

In [ ]:
iterador.__next__()

In [ ]:
def is_palindrome(num):
    
    r = str(abs(num)) if type(int) else num

    if len(r) < 2:
        return False
    
    elif len(r) %2 == 0:
        size = len(r)//2
        a = r[:size]
        b = r[size:][::-1]
        
        return a==b
        

    else:
        size = len(r)//2
        a = r[:size]
        b = r[size+1:][::-1]
        
        return a==b
        
listado = list()

for i in range(-1000,1000):

    if (is_palindrome(i)):
        listado.append(i)

print(listado)

# <center> TIPOS DE INSTANCIA</center>

# <center> SINCRONISMO - HILOS - PROCESOS - ASINCRONISMO</center>

<center>
    <img width="50%" lign="center" src="ejecuciones.png">
</center>

# <center> EJECUCION **SINCRONA**</center>

In [2]:
%%time

from time import sleep
espera = 5

def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo


datos = []

for tiempo_espera in range(1,espera + 1):
    datos.append(dormir_sincrono(tiempo_espera))
    
datos

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
CPU times: user 18.4 ms, sys: 1.53 ms, total: 19.9 ms
Wall time: 15 s


[1, 2, 3, 4, 5]

# <center> EJECUCION **ASINCRONA**</center>

In [3]:
from datetime import datetime
from time import sleep
import asyncio

async def corrutina(tiempo:int) -> int:
    print('corriendo tarea de tiempo',tiempo)
    await asyncio.sleep(tiempo)
    return tiempo

datos = []
resul = []

inicia = datetime.now()

# crea el listado de corrutinas
for tiempo_espera in range(1,espera + 1):
    datos.append(asyncio.gather(corrutina(tiempo_espera)))
    
# recupera los valores de las corrutinas
for corrutina in datos:
    valor = await corrutina
    resul.append(valor[0])

finaliza = datetime.now()
print(finaliza.second - inicia.second) 
resul

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
5


[1, 2, 3, 4, 5]

# <center> EJECUCION **HILO**</center>
## <center> Este módulo construye interfaces de hilado de alto nivel sobre el módulo de más bajo nivel _thread. 

In [4]:
%%time
# libreria para procesamiento en paralelo
from threading import Thread

# libreria que permite guardar los valores retornados en una cola
from queue import Queue as cola_classica

# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

def almacenar_en_cola(f):
    def wrapper(*args):
        almacenar_resultados.put(f(*args))
    return wrapper

@almacenar_en_cola
def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

def obtener_resultado_cola_tarea(cola):
    
    valores = []
    for indice_tarea in range(cola.qsize()):
        valores.append(cola.get(indice_tarea))
    
    return valores


listado_instancias = []
   
    
for tiempo_espera in range(1,espera + 1):
  
    instancia = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,))

    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
obtener_resultado_cola_tarea(almacenar_resultados)

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
CPU times: user 0 ns, sys: 28.3 ms, total: 28.3 ms
Wall time: 5.02 s


[1, 2, 3, 4, 5]

# <center> EJECUCION **PROCESO**</center>
## <center> Este módulo permite crear procesos (spawning) utilizando una API similar al módulo threading. ofrece concurrencia tanto local como remota. permite aprovechar al máximo múltiples procesadores en una máquina determinada.

In [5]:
%%time

# importa las librerias que permiten usar la computacion paralela
from multiprocessing import Process

# libreria que permite guardar los valores retornados en una cola
from multiprocessing import Queue as cola_multiproceso

# instancia de la cola para adicionar los resultados
almacenar_resultados_multiproceso = cola_multiproceso()

def almacenar_en_cola_multiproceso(f):
    def wrapper(*args):
        almacenar_resultados_multiproceso.put(f(*args))
    return wrapper

@almacenar_en_cola_multiproceso
def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

def obtener_resultado_cola_tarea_while(cola):
    
    valores = []
    while not cola.empty():
        result = cola.get()
        valores.append(result)    
    return valores
        

listado_instancias = []

for tiempo_espera in range(1,espera + 1):
  
    instancia = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,))
    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()


obtener_resultado_cola_tarea_while(almacenar_resultados_multiproceso)

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo3
4 
5
CPU times: user 68.6 ms, sys: 74.1 ms, total: 143 ms
Wall time: 5.2 s


[1, 2, 3, 4, 5]

# <center> EJECUCION 1800 TAREAS DE 2 SEGUNDOS, TOMARIA 1 HORA EN EJECUCION SINCRONA</center>

## <center> THREAD</center>

In [12]:
%%time

import random

limite_tareas = 1800

almacenar_resultados = cola_classica()
listado_instancias = []

@almacenar_en_cola
def dormir_sincrono(tiempo:int) -> str:    
    sleep(tiempo)
    return random.randint(0,limite_tareas)

    
for tiempo_espera in range(limite_tareas):
  
    instancia = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(2,))

    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(len(resultado))
print(resultado)

1800
[634, 15, 1546, 1702, 1408, 211, 1561, 1488, 377, 638, 1690, 1510, 866, 995, 504, 1473, 250, 602, 1212, 577, 143, 603, 1506, 1340, 398, 1771, 745, 122, 1782, 1239, 439, 1461, 243, 477, 1567, 158, 1323, 1015, 279, 1586, 1395, 1259, 1108, 1478, 556, 69, 925, 1206, 1782, 204, 1577, 1613, 1288, 407, 1680, 619, 1148, 303, 1299, 1222, 702, 331, 1377, 357, 51, 2, 1283, 200, 839, 671, 828, 1642, 21, 373, 1721, 220, 444, 1415, 1314, 202, 577, 0, 173, 1028, 239, 917, 178, 1784, 535, 1544, 584, 836, 995, 468, 1134, 1338, 519, 1180, 1043, 165, 703, 1083, 1137, 189, 1007, 983, 960, 843, 518, 9, 842, 1776, 1747, 1521, 1180, 19, 467, 168, 1753, 1706, 1667, 1795, 1051, 1607, 744, 1629, 181, 1523, 1327, 1396, 661, 1794, 1300, 1154, 472, 1575, 97, 53, 69, 733, 1569, 1609, 1313, 925, 522, 479, 1646, 926, 361, 465, 1121, 630, 495, 725, 1668, 538, 1545, 1213, 190, 152, 235, 576, 1315, 1053, 898, 702, 848, 1223, 1504, 137, 276, 1021, 1550, 491, 1394, 481, 732, 1317, 160, 948, 1678, 277, 1676, 1259, 130

## <center> PROCESS</center>

In [13]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados_multiproceso = cola_multiproceso()
listado_instancias = []


@almacenar_en_cola_multiproceso
def dormir_sincrono(tiempo:int) -> str:    
    sleep(tiempo)
    return random.randint(0,limite_tareas)
        

for tiempo_espera in range(limite_tareas):
  
    instancia = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(2,))
    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

resultado = obtener_resultado_cola_tarea_while(almacenar_resultados_multiproceso)
print(len(resultado))
print(resultado)

1800
[18, 145, 945, 456, 64, 1342, 1228, 1194, 575, 1319, 1049, 1642, 1158, 1493, 527, 1084, 1317, 1402, 712, 263, 1187, 1684, 247, 1554, 488, 337, 97, 219, 187, 1443, 484, 130, 446, 1312, 1408, 1335, 441, 1362, 22, 673, 1054, 1221, 1534, 394, 160, 92, 264, 165, 1176, 758, 1771, 926, 70, 1075, 1220, 1088, 588, 348, 1296, 177, 236, 1562, 365, 917, 424, 14, 438, 1760, 886, 315, 572, 1234, 1507, 585, 1122, 1595, 320, 29, 298, 1519, 421, 256, 1407, 807, 1525, 1565, 1782, 1197, 55, 827, 864, 1084, 221, 923, 1057, 1762, 1179, 1486, 1558, 1444, 1434, 635, 1587, 74, 1737, 893, 255, 1552, 258, 650, 1756, 1006, 1502, 870, 673, 209, 979, 113, 1411, 632, 1668, 1061, 1371, 824, 774, 491, 1548, 57, 998, 103, 1736, 538, 1761, 571, 1058, 1071, 583, 78, 525, 549, 935, 567, 435, 763, 797, 1313, 994, 61, 698, 533, 1772, 619, 1308, 635, 1127, 1228, 966, 1493, 590, 1447, 985, 370, 577, 903, 834, 644, 1729, 1556, 270, 1360, 22, 32, 1038, 1168, 1754, 1163, 222, 1773, 810, 391, 62, 1530, 1528, 1448, 920, 90, 

## <center> ASYNCHRONOUS</center>

In [15]:
async def corrutina(tiempo:int) -> str:    
    await asyncio.sleep(tiempo)
    return random.randint(0, limite_tareas)

almacenar_resultados = []
listado_instancias = []

inicia = datetime.now()

# crea el listado de corrutinas
for tiempo_espera in range(limite_tareas):
    listado_instancias.append(asyncio.gather(corrutina(2)))
    
# recupera los valores de las corrutinas
for corrutina in listado_instancias:
    valor = await corrutina
    almacenar_resultados.append(valor[0])

finaliza = datetime.now()
print(finaliza.second - inicia.second) 

resultado = almacenar_resultados
print(len(resultado))
print(resultado)

2
1800
[263, 1364, 1398, 389, 1656, 146, 700, 764, 1134, 15, 1069, 1515, 1382, 1061, 357, 1517, 656, 256, 548, 449, 1418, 694, 948, 509, 1464, 1236, 774, 1122, 1151, 103, 1026, 1648, 942, 515, 194, 1454, 1758, 593, 1080, 1072, 800, 887, 86, 1461, 731, 165, 908, 318, 168, 1468, 1780, 1721, 641, 173, 161, 48, 313, 1423, 568, 349, 1118, 252, 322, 1379, 1676, 764, 425, 1077, 495, 140, 758, 175, 559, 113, 257, 820, 1502, 311, 1553, 1658, 1523, 1703, 1553, 1075, 263, 407, 528, 937, 1348, 820, 1152, 473, 796, 996, 1650, 127, 362, 1486, 1540, 1356, 1298, 616, 1734, 822, 1797, 1635, 224, 674, 1381, 628, 1452, 1558, 1229, 1505, 88, 745, 776, 1740, 1541, 609, 125, 978, 943, 1014, 1226, 68, 1490, 132, 440, 0, 1217, 610, 1259, 1697, 612, 1343, 120, 243, 1276, 1626, 1195, 1355, 136, 651, 341, 252, 988, 423, 1034, 532, 627, 1125, 1057, 1133, 114, 731, 318, 1312, 475, 1186, 973, 1176, 844, 1215, 16, 231, 446, 989, 752, 1791, 504, 899, 805, 131, 1121, 103, 14, 395, 951, 1680, 263, 508, 387, 274, 498, 5

# <center> FUNCIONES PARA LOS HILOS Y PROCESOS</center>

In [ ]:
def generar_instancia(tipo_tarea=None, funcion_a_procesar=None, **kw):
    
    # convierte los argumentos a tupla ya que los argumentos deben ser de tipo tupla
    argumentos = tuple(kw.values())
    
    # tipos de Instancia que se van a usar
    tipo_instancia = {
            'hilos':Thread,
            'procesos':Process
            }
        
    # crea la instancia
    tipo_ejecucion = tipo_instancia.get(tipo_tarea, Thread)
    
    # retorna la instancia con los argumentos a evaluar
    return tipo_ejecucion(name='test_ejecucion', target=funcion_a_procesar, args=argumentos)


def iniciar_instancia(instancia):
    
    # muestra el nombre del hilo o del proceso
    print('tarea inicializada ', instancia.name)
    
    # inicializa la ejecucion de la instruccion invocada en el hilo o proceso
    instancia.start()
    
    return instancia

def finalizar_instancia(instancia):

    # garantiza que se finaliza la funcion invocada, antes de que se continue con una nueva instruccion
    instancia.join()
    
    # muestra el nombre del hilo o del proceso
    print('tarea Finalizada', instancia.name)
    
    return instancia

# <center> HILOS</center>
## <center>Es una linea de ejecucion de un proceso, Para ejecuciones a gran escala permite delegar acciones en tareas mas pequeñas</center>

## <center> un Lock es un mecanismo de sincronización para forzar el acceso a una sección de código en un programa. por ejemplo </center>

# <center> COLA PARA ADICIONAR RESULTADOS HILOS</center>

In [ ]:
# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

def almacenar_en_cola(f):
    def wrapper(*args):
        almacenar_resultados.put(f(*args))
    return wrapper

@almacenar_en_cola
def retornar_lista(numero=100):
    return [i for i in range(numero)]

def obtener_resultado_cola_tarea(cola):
    
    for indice_tarea in range(cola.qsize()):
        print(cola.get(indice_tarea))

In [ ]:
listado_instancias = []
   
    
for cantidad_instancias in range(5):
  
    instancia = generar_instancia(tipo_tarea='hilos', funcion_a_procesar=retornar_lista, numero=20)

    ejecutar_tarea = iniciar_instancia(instancia)
   
    listado_instancias.append(ejecutar_tarea)


for tipo_instancia in listado_instancias:
    
    finalizar_instancia(tipo_instancia)

obtener_resultado_cola_tarea(almacenar_resultados)

# <center> PROCESOS</center>
## <center> Process-based parallelism, estan compuestos de multiples hilos, utilizan todos los nucleos del procesador que se puedan usar.</center>

# <center> COLA PARA ADICIONAR RESULTADOS PROCESOS</center>

In [ ]:
# instancia de la cola para adicionar los resultados
almacenar_resultados_multiproceso = cola_multiproceso()

def almacenar_en_cola_multiproceso(f):
    def wrapper(*args):
        almacenar_resultados_multiproceso.put(f(*args))
    return wrapper

@almacenar_en_cola_multiproceso
def retornar_lista_proceso(numero=20):
    return [chr(i + 64)  for i in range(numero)]

def obtener_resultado_cola_tarea_while(cola):
    
    while not cola.empty():
        result = cola.get()
        print (result)

In [ ]:
listado_instancias = []

for cantidad_instancias in range(5):
  
    instancia = generar_instancia(tipo_tarea='procesos', funcion_a_procesar=retornar_lista_proceso, numero=10)

    ejecutar_tarea = iniciar_instancia(instancia)
   
    listado_instancias.append(ejecutar_tarea)


for tipo_instancia in listado_instancias:
    
    finalizar_instancia(tipo_instancia)

obtener_resultado_cola_tarea_while(almacenar_resultados_multiproceso)

# <center> HILOS I/O VS PROCESOS I/O</center>

In [ ]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

from multiprocessing.pool import Pool
from multiprocessing.pool import ThreadPool

cantidad = 100_000
workers = 4

def lectura_io(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)

## <center> HILOS I/O</center>

In [ ]:
%%time
with ThreadPoolExecutor(max_workers=workers) as concurrent_futures:        
    concurrent_futures.map(lectura_io, range(cantidad))

In [ ]:
%%time
valores = []

with ThreadPool(processes=workers) as multiprocessing:        
    print(len(multiprocessing.map(lectura_io, range(cantidad))))

## <center> PROCESOS I/O</center>

In [ ]:
%%time
with ProcessPoolExecutor(max_workers=workers) as concurrent_futures:        
    print(len(list(concurrent_futures.map(lectura_io, range(cantidad)))))

In [ ]:
%%time
with Pool(processes=workers) as multiprocessing:
    datos = multiprocessing.map(lectura_io, range(cantidad))
    print(len(multiprocessing.map(lectura_io, range(cantidad))))

# <center> HILOS CPU VS PROCESOS CPU</center>

In [ ]:
import math

PRIMES = range(1_099_726_899_280_000, 1_099_726_899_280_999) 

def is_prime(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

## <center> HILOS CPU</center>

In [ ]:
%%time
valores = []
with ThreadPoolExecutor(max_workers=workers) as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        valores.append((number, prime))

print('cantidad de valores procesados', len(valores))

In [ ]:
%%time
valores = []
with ThreadPool(processes=workers) as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        valores.append((number, prime))

print('cantidad de valores procesados', len(valores))

## <center> PROCESOS CPU</center>

In [ ]:
%%time

valores = []
with ProcessPoolExecutor(max_workers=workers) as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
        valores.append((number, prime))

print('cantidad de valores procesados', len(valores))

In [ ]:
%%time

valores = []
with Pool(processes=workers) as executor:
    
    valores = executor.imap_unordered(is_prime, PRIMES)


print('cantidad de valores procesados', valores)

In [ ]:
%%time

valores = []
with Pool(processes=workers) as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
        valores.append((number, prime))

print('cantidad de valores procesados', len(valores))

In [ ]:


async def main():
    loop = asyncio.get_running_loop()
    cantidad = 100

    ## Options:
    inicia = datetime.now()
    # 1. Run in the default loop's executor:
    for i in range(cantidad):
        result = await loop.run_in_executor(
            None, blocking_io)
    print('default thread pool', result)
    finaliza = datetime.now()
    print(finaliza.second - inicia.second ) 
    
    
    inicia = datetime.now()
    # 2. Run in a custom thread pool:
    for i in range(cantidad):
        with concurrent.futures.ThreadPoolExecutor() as pool:
            result = await loop.run_in_executor(
                pool, blocking_io)
    print('custom thread pool', result)
    finaliza = datetime.now()
    print(finaliza.second - inicia.second ) 
    
    inicia = datetime.now()
    # 3. Run in a custom process pool:
    for i in range(cantidad):
        with concurrent.futures.ProcessPoolExecutor() as pool:
            result = await loop.run_in_executor(
                pool, cpu_bound)
    print('custom process pool', result)
    finaliza = datetime.now()
    print(finaliza.second - inicia.second ) 
await main()

# <center> POOL DE PROCESOS E HILOS</center>
## <center>Ofrece un medio conveniente de paralelizar la ejecución de una función a través de múltiples valores de entrada, distribuyendo los datos de entrada a través de procesos (paralelismo de datos).</center>

In [ ]:
from multiprocessing.pool import Pool

def calcular_cuadrado(x):
    return x*x

### <center> APPLY - PROCESOS</center>

In [ ]:
pool = Pool(processes=2)
pool_apply = pool.apply(calcular_cuadrado, [5])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

### <center> APPLY - HILOS</center>

In [ ]:
pool = ThreadPool(processes=2)
pool_apply = pool.apply(calcular_cuadrado, [7])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

### <center> APPLY_ASYNC</center>

In [ ]:
pool = Pool(processes=2)
pool_apply_async = pool.apply_async(calcular_cuadrado, [3])
pool.close()
pool.join()
pool.close()
pool_apply_async.get()

### <center> MAP_ASYNC</center>

In [ ]:
pool = Pool(processes=2)
pool_map_async = pool.map_async(calcular_cuadrado, range(0,20,3))
pool.close()
pool.join()
pool.close()
pool_map_async.get()

### <center> MAP</center>

In [ ]:
pool = Pool(processes=2)
pool_map = pool.map(calcular_cuadrado, range(0,20,5))
pool.close()
pool.join()
pool.close()
pool_map

### <center> IMAP</center>

In [ ]:
pool = Pool(processes=2)
pool_imap_async = pool.imap(calcular_cuadrado, range(0,20,4))
pool.close()
pool.join()
pool.close()
list(pool_imap_async)

### <center> IMAP_UNORDERED</center>

In [ ]:
pool = Pool(processes=2)
pool_imap_unordered = pool.imap_unordered(calcular_cuadrado, range(0,20,2))
pool.close()
pool.join()
pool.close()
list(pool_imap_unordered)

### <center> STARMAP</center>

In [ ]:
pool = Pool(processes=2)
pool_starmap = pool.starmap(calcular_cuadrado, [[1],[2],[3],[4],[5]])
pool.close()
pool.join()
pool.close()
list(pool_starmap)

### <center> STARMAP_ASYNC</center>

In [ ]:
pool = Pool(processes=2)
pool_starmap_async = pool.starmap_async(calcular_cuadrado, [[1],[2],[3],[4],[5]])
pool.close()
pool.join()
pool.close()
pool_starmap_async.get()

# <center> PROCESAMIENTO MASIVO</center>
## <p> Lanzamiento de tareas paralelas, El módulo concurrent.futures provee una interfaz de alto nivel para ejecutar **invocables de forma asincrónica**. La ejecución asincrónica se puede realizar mediante hilos, usando ThreadPoolExecutor, o procesos independientes, mediante ProcessPoolExecutor. Ambos implementan la misma interfaz, que se encuentra definida por la clase abstracta Executor.</p>

In [ ]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

# tipos de Instancia que se van a usar
tipo_instancia = {
        'hilos':Thread,
        'procesos':Process
        }

# adiciona nuevas key/values al diccionario
tipo_instancia['multihilos'] = ThreadPoolExecutor
tipo_instancia['multiproceso'] = ProcessPoolExecutor

tipo_instancia

In [ ]:
import random

def generar_lista(numero = 22):
    
    return [random.randint(-100, i**2) for i in range(numero)]
    
def imprimir_multi_datos(lista):
    
    for valores in lista:
        print(valores)
        
def generar_pool_iniciar_y_finalizar_multi_instancia(tipo_tarea=None, funcion_a_procesar=None, numero = 22):
    
    # crea la instancia de ejecucion
    tipo_ejecucion = tipo_instancia.get(tipo_tarea, ThreadPoolExecutor)
    
    # especifica el numero de hilos o procesos que se quieren usar como base, workers basado en el total de CPUs de la maquina
    pool = tipo_ejecucion(max_workers=nucleos_totales)
    
    # recupera la informacion de la tarea que se ejecuta
    response = pool.submit(funcion_a_procesar, numero)
    
    # finaliza el pool
    pool.shutdown()
    
    # obtiene la respuesta de la tarea
    return response.result()

# <center> MULTI HILOS ThreadPoolExecutor, usa un grupo de hilos para ejecutar llamadas de forma asincrónica.</center>
# <center> interruptores rápidos b/w que dan la ilusion de multi tarea</center>

In [ ]:
listado = []
for i in range(5):
    instancia = generar_pool_iniciar_y_finalizar_multi_instancia(tipo_tarea='multihilos', funcion_a_procesar=generar_lista, numero= i + 1)
    # almacena las tareas ejecutadas
    listado.append(instancia)

imprimir_multi_datos(listado)

# <center> MULTI PROCESOS ProcessPoolExecutor, usa un grupo de procesos para ejecutar llamadas de forma asíncrona</center>

In [ ]:
listado = []
for i in range(5):
    instancia = generar_pool_iniciar_y_finalizar_multi_instancia(tipo_tarea='multiprocesos', funcion_a_procesar=generar_lista, numero= i + 1)
    # almacena las tareas ejecutadas
    listado.append(instancia)

imprimir_multi_datos(listado)

# <center> MULTI HILOS Y MULTIPROCESOS CON MULTIPLES ARGUMENTOS USANDO **MAP**</center>

In [ ]:
def cuadrado(x=9):
    
    return x**2

def multi_argumentos(*parametros, tipo_tarea=None, funcion_a_procesar=None):
    
    # convierte los parametros tipo tupla a lista
    argumentos = list(parametros[0])
    
    # crea la instancia de ejecucion
    tipo_ejecucion = tipo_instancia.get(tipo_tarea, ThreadPoolExecutor)
    
    # asigna los workers basado en el total de CPUs de la maquina
    pool = tipo_ejecucion(max_workers=nucleos_totales)
        
    # obtiene los resultados de cada argumento en la funcion  
    response = pool.map(cuadrado, argumentos)
        
    # obtiene la respuesta de la tarea
    return [valor for valor in response]

print(multi_argumentos([1,2,3,4],tipo_tarea='multihilos', funcion_a_procesar=cuadrado))
print(multi_argumentos([5,6,7,8,9],tipo_tarea='multiprocesos', funcion_a_procesar=cuadrado))

In [ ]:
import threading
import time


def thread_x():
    print('Start ', threading.current_thread().name)
    time.sleep(1)
    print('Finish ', threading.current_thread().name)


def thread_y():
    print('Start ', threading.current_thread().name)
    time.sleep(1)
    print('Finish ', threading.current_thread().name)

x = threading.Thread(target=thread_x, name='Thread-X', daemon = True)
y = threading.Thread(target=thread_y, name='Thread-Y')


x.start()
y.start()

x.join()
y.join()

# <center> ASINCRONISMO - ASYNCIO</center>

# <center> EVENT LOOP</center>

<p align="center" width="100%">
    <img width="100%" src="event-loop.png">
</p>

# <p> asyncio es una biblioteca para escribir código concurrente utilizando la sintaxis async/await.Es utilizado como base en múltiples frameworks asíncronos de Python y provee un alto rendimiento en redes y servidores web, bibliotecas de conexión de base de datos, colas de tareas distribuidas, etc. </p>

- asyncio provee un conjunto de APIs de alto nivel para:

- ejecutar corutinas de Python de manera concurrente y tener control total sobre su ejecución;

- realizar redes E/S y comunicación entre procesos(IPC);

- controlar subprocesos;

- distribuir tareas a través de colas;

- sincronizar código concurrente;


- Adicionalmente, existen APIs de bajo nivel para desarrolladores de bibliotecas y frameworks para:

- crear y administrar bucles de eventos, los cuales proveen APIs asíncronas para redes, ejecutando subprocesos, gestionando señales del sistema operativo, etc;

- implementar protocolos eficientes utilizando transportes;

- Bibliotecas puente basadas en retrollamadas y código con sintaxis async/wait.

In [ ]:
import asyncio

# esta seria la forma de ejecutar en un IDE, dado que  jupyter ya esta corriendo un event loop
# siempre es necesario ejecutar una funcion asincrona de esta manera 
#asyncio.run(funcion_asincrona())

# <center> TODA FUNCION ASINCRONA DEBE SER PRECEDIDA POR LA PALABRA **ASYNC**</center>

In [ ]:
async def listado(limite = 10):
    
    return [random.randrange(-100, 100) for x in range(limite)]

In [ ]:
def ejecuta_funcion_listado(limite = 15):
    return listado(limite)

In [ ]:
async def ejecuta_funcion_asincrona(limite = 20):
    return await listado(limite)

# <center> TODA VARIABLE O FUNCION QUE LLAME UNA FUNCION ASYNCRONA DEBE SER PRECEDIDA POR LA PALABRA **AWAIT**</center>

In [ ]:
ejecuta_funcion_listado()

In [ ]:
respuesta = listado()
respuesta

In [ ]:
await respuesta

In [ ]:
await listado()

In [ ]:
await ejecuta_funcion_listado()

In [ ]:
print(await ejecuta_funcion_asincrona())

In [ ]:
lista = []

for i in range(10):
    
    lista.append(await listado())

lista

# <center> **FUNCIONES ASINCRONAS**</center>

In [ ]:
async def Primero(limite=5):
    print("Inicio funcion Primero")
    await asyncio.sleep(2)
    print("Final funcion Primero")
    
    return [random.randrange(-100, 0) for x in range(limite)]

async def Ultimo(limite=5):
    print("Comenzar funcion ultimo")
    await asyncio.sleep(2)
    print("Terminar funcion ultimo")
    
    return [random.randrange(0, 100) for x in range(limite)]

# <center> **GATHER**</center>
## <center> **RECIBE UNA LISTA DE TAREAS Y RETORNA UNA LISTA DE RESULTADOS**</center>
## <center> **TIENE OPCIONES ESPECÍFICAS PARA EL MANEJO DE ERRORES Y CANCELACIONES.**</center>

In [ ]:
# El siguiente ejemplo muestra cómo esperar a que se completen varias tareas asincrónicas.

from asyncio import gather

async def iniciar_gather():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    
    return await gather(*tareas)
    
await iniciar_gather()

# <center> **WAIT_FOR**</center>
## <center> **PERMITE DEFINIR EL MAXIMO TIEMPO DE ESPERA DE UNA TAREA**</center>

In [ ]:
# El siguiente ejemplo demuestra cómo podemos utilizar un tiempo de espera para evitar esperar indefinida a que finalice una tarea asincrónica.

from asyncio import wait_for

async def iniciar_wait_for():
    try:
        return await wait_for(Primero(), timeout=1)
    except asyncio.TimeoutError:
        print("la ejecucion supero el tiempo de espera!")
        

await iniciar_wait_for()

# <center> **AS_COMPLETED**</center>
## <center> **ES SIMILIAR A GATHER, PERO RETORNA FUTUROS, LOS RESULTADOS SON RETORNADOS EN EL ORDEN QUE ESTAN LISTOS**</center>

In [ ]:
# El siguiente ejemplo demuestra cómo as_complete, completará la primera tarea, seguida de la siguiente más rápida y la siguiente hasta que se completen todas las tareas.

from asyncio import as_completed

async def iniciar_as_completed():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    counter = 0
    
    for future in as_completed(tareas):
        n = "la tarea mas rapida" if counter == 0 else "la siguiente tarea mas rapida"
        counter += 1
        result = await future
        print(f"{n} obtuvo el resultado: {result}")

await iniciar_as_completed()

# <center> **CREATE_TASK**</center>

In [ ]:
# El siguiente ejemplo demuestra cómo convertir una rutina en una tarea y programarla en el bucle de eventos.

from asyncio import create_task

async def iniciar_create_task():
    
    tarea = create_task(Primero())
    print(tarea)
    
    await asyncio.sleep(2)
    print("MAS PROCESOS!")
    
    await asyncio.sleep(3)
    print(sum(tarea))
    
    return tarea

await iniciar_create_task()

# <center> **POOLS**</center>
# <center> **GET_RUNNING_LOOP**</center>

## <center> **ASYNCIO Event Loop - MULTI HILO**</center>

In [ ]:
# importa la libtreria
from asyncio import get_running_loop
# importa librerias para trabajar concurrencia
from concurrent.futures import ThreadPoolExecutor

def blocking_io(value = 23, otros = 10):
    # File operations (such as logging) can block the
    # event loop: run them in a thread pool.
    with open("/dev/urandom", "rb") as f:
        return f.read(value + otros)
    
async def asyncio_multi_hilo():
    
    loop = get_running_loop()
    pool = ThreadPoolExecutor()
    
    # tipo de pool, funcion, parametros
    return await loop.run_in_executor(pool, blocking_io, 50, 70)

await asyncio_multi_hilo()

## <center> **ASYNCIO - MULTI PROCESO**</center>

In [ ]:
from concurrent.futures import ProcessPoolExecutor

def cpu_bound(value = 23, otros = 10):
    # CPU-bound operations will block the event loop:
    # in general it is preferable to run them in a
    # process pool.
    return  sum(i * i for i in range(10 ** 7)) // (value + otros)

    
async def asyncio_multi_proceso():
    
    loop = get_running_loop()
    pool = ProcessPoolExecutor()
    
    # tipo de pool, funcion, parametros
    return await loop.run_in_executor(pool, cpu_bound, 123, 100)

await asyncio_multi_proceso()

 # TABLA DE DEFINICIONES
 https://pharos.sh/concurrencia-en-python/#:~:text=al%20mismo%20tiempo.-,Simultaneidad%20vs%20paralelismo,acelerar%20el%20proceso%20de%20c%C3%A1lculo.
 

| CONCURRENCIA O SIMULTANEIDAD  | PARALELISMO |
| --- | --- |
| Es la ejecución de tareas al mismo tiempo, Cuando dos o más eventos son concurrentes, significa que están sucediendo al mismo tiempo. | Se logra cuando se realizan múltiples cálculos u operaciones al mismo tiempo o en paralelo con el objetivo de acelerar el proceso de cálculo. | 
| Solo tiene lugar en un procesador. | Utiliza múltiples procesadors para realizar tareas en paralelo. | 
| es la tarea de ejecutar y administrar múltiples cálculos al mismo tiempo. | es la tarea de ejecutar múltiples cálculos simultáneamente. |
| se logra a través de la operación de entrelazado de procesos en la unidad central de procesamiento (CPU) o, en otras palabras, mediante el cambio de contexto. | se logra a través de múltiples unidades centrales de procesamiento (CPU) |
| se puede realizar utilizando una sola unidad de procesamiento. | necesita múltiples unidades de procesamiento. |
| aumenta la cantidad de trabajo terminado a la vez. |  mejora el rendimiento y la velocidad computacional del sistema.|
| trata muchas cosas simultáneamente. | hace muchas cosas simultáneamente. |
|  es el enfoque de flujo de control no determinista.| es un enfoque de flujo de control determinista. |
|  la depuración es muy difícil. | la depuración también es difícil pero mas simple que la concurrencia. |



| HILO O SUBPROCESO | PROCESO | TAREA |
| --- | --- | --- |
| cada tarea ejecutada en un proceso es un subproceso. | Es un trabajo o una instancia de un programa calculado que se puede ejecutar. | Es un conjunto de instrucciones de programa que se cargan en la memoria.|
| Es la unidad de ejecución más pequeña que se puede realizar en una computadora. | Un hilo solo puede pertenecer a un proceso, pero un proceso puede tener múltiples hilos. |
| pueden acceder a los datos de otros subprocesos  | funcionan de forma aislada | |
| comparten memoria con otros subprocesos | cada proceso tiene su propia asignación de memoria. | |
||es un proceso liviano: en comparación con un proceso|||
| un subproceso genera menos carga en el sistema operativo para crear, mantener y administrar, lo que significa que el costo o la sobrecarga del subproceso es relativamente pequeño.|||
| El hilo no tiene espacio de direcciones, y el hilo está contenido en el espacio de direcciones del proceso.|||
| Todos los hilos comparten la memoria y los recursos del proceso. |||


| HILOS | PROCESOS | ASINCRONISMO |
| --- | --- | --- |
| Implementa concurrencia a través de hilos de aplicación | Implementa la concurrencia usando procesos del sistema | Construye aplicaciones concurrentes que utilizan co-rutinas. Utiliza un enfoque de un solo hilo y un solo proceso en el que partes de una aplicación cooperan para cambiar tareas explícitamente en momentos óptimos.|
|Multihilo es el proceso en el que multiples hilos se ejecutan al mismo tiempo en un proceso.|El multiproceso es permitir que se realicen múltiples procesos al mismo tiempo,  utilizando  uno o mas procesadores.|